# Part I. ETL Pipeline for Pre-Processing the Files

## PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
 import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())
filepath = os.getcwd() + '/event_data'
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = [] 
    
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile:  
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        for line in csvreader:
            full_data_rows_list.append(line) 
            
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


 

## The CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
import cassandra

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except exception as e:
    print(e)

#### Create Keyspace

In [6]:
try:
    session.execute(""" CREATE KEYSPACE IF NOT EXISTS music 
    with replication = 
    {'class': 'SimpleStrategy', 'replication_factor': 1} """)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('music')
except Exception as e:
    print(e)


### Create tables to run the following queries in Apache Cassandra, we model the database tables on the queries we want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




Table artist_song_by_session will be created by using column 'sessionId' and column 'itemInSession' as a compond partition key because the queries will filter by these columns.

In [14]:
try:
    session.execute ("create table if not exists artist_song_by_session"
    "(sessionId INT, \
    itemInSession INT, \
    artist TEXT, \
    song TEXT, \
    length FLOAT, \
    PRIMARY KEY(sessionId, itemInSession))")
    
except Exception as e:
    print(e)
                    

In [25]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        
        query = "insert into artist_song_by_session(sessionId, itemInSession, artist, song, length)"
        query = query + "values(%s, %s, %s,%s, %s)"
        
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [28]:
query = "select artist, song, length from artist_song_by_session where sessionId = 338 and itemInSession = 4"

try:
    r = session.execute(query)
except Exception as e:
    print(e)

tab = pd.DataFrame(r._current_rows)
tab

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

Table users_listening_to_songs_by_session will be created by using column 'userId' and column 'sessionId' as a compond partition key because the queries will filter by these columns and column 'itemInSession' will be the clustering column.

In [30]:
try:
    session.execute ("create table if not exists users_listening_to_songs_by_session"
    "(userId INT, \
    sessionId INT, \
    itemInSession INT, \
    artist TEXT, \
    song TEXT, \
    firstName TEXT, \
    lastName TEXT, \
    PRIMARY KEY( (userId, sessionId), itemInSession))")
    
except Exception as e:
    print(e)


In [32]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "insert into users_listening_to_songs_by_session(userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "values(%s, %s, %s,%s, %s, %s, %s)"
        
        try:
            session.execute(query, ( int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        except Exception as e:
                print(e)

In [34]:
query = "select artist, song, firstName, lastName from users_listening_to_songs_by_session where userid = 10 and sessionid = 182"

try:
    r = session.execute(query)
except Exception as e:
    print(e)
    
tab = pd.DataFrame(r._current_rows)
tab                

,artist,song,firstname,lastname
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


Table users_listened_to_All_Hands_Against_His_Own will be created to show all users first and last name who listened to the song 'All Hands Against His Own' by using column 'song' as a partition key and column 'userid' as a clustering column because the queries will filter by these columns.

In [35]:
try:
    session.execute ("create table if not exists users_listened_to_All_Hands_Against_His_Own"
    "(song TEXT, \
    userId INT, \
    firstName TEXT, \
    lastName TEXT, \
    PRIMARY KEY (song, userid))")
    
except Exception as e:
    print(e)

In [37]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "insert into users_listened_to_All_Hands_Against_His_Own( song, userid, firstName, lastName)"
        query = query + "values(%s, %s, %s, %s)"
        
        try:
            session.execute(query, (line[9], int(line[10]), line[1], line[4]))
        except Exception as e:
                print(e)

In [38]:
query = "select firstName, lastName from users_listened_to_All_Hands_Against_His_Own where song = 'All Hands Against His Own'" 

try:
    r = session.execute(query)
except Exception as e:
    print(e)
tab = pd.DataFrame(r._current_rows)
tab

,firstname,lastname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [39]:
try:
        session.execute("drop table artist_song_by_session")
        session.execute("drop table users_listening_to_songs_by_session")
        session.execute("drop table users_listened_to_All_Hands_Against_His_Own")
        
except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [79]:
session.shutdown()
cluster.shutdown()